In [17]:
import requests as rq
import time
import matplotlib.pyplot as plt
from datetime import datetime
URL = 'localhost:8080'
DAY = 24 * 60 * 60 * 1000

message = rq.get(URL+'/devices').json()
devices = message['mac_addresses']
print(devices)
for device in devices:
    now = round(time.time() * 1000)
    message = rq.get(URL + f'/device/{device}', params = {'from': now - DAY, 'to': now}).json()
    if message['timestamps']:
        dates_battery_levels = []
        battery_levels = []
        for timestamp, battery_level in zip(message['timestamps'], message['battery_levels']):
            if battery_level != "null":
                dates_battery_levels.append(datetime.fromtimestamp(timestamp/1000))
                battery_levels.append(battery_level)
        dates_powers = []
        powers = []
        for timestamp, power in zip(message['timestamps'], message['power_plugged']):
            if power != 'null':
                dates_powers.append(datetime.fromtimestamp(timestamp/1000))
                powers.append(power)

        _, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].plot_date(dates_battery_levels, battery_levels)
        ax[0].set_xticklabels(dates_battery_levels, rotation=70)
        ax[0].set_xlabel('Time')
        ax[0].set_ylabel('Battery level')
        ax[0].set_title('Battery level')

        ax[1].plot_date(dates_powers, powers)
        ax[1].set_xticklabels(dates_powers, rotation=70)
        ax[1].set_xlabel('Time')
        ax[1].set_ylabel('Power plugged')
        ax[1].set_title('Power plugged')
if devices:
    rq.delete(URL + f'/device/{devices[-1]}')